In [3]:
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
conn_data = ({
    'USERNAME': 'postgres',
    'DB_NAME': 'iowa_liquor_sales',
    'PASSWORD': 'postgres',
    'HOST': 'localhost',
    'PORT': '5432'
})

In [5]:
sale_new = '''
CREATE TABLE sale_new(
    invoice VARCHAR(30) PRIMARY KEY,
    store_name VARCHAR(50) NOT NULL,
    vendor_num INT NOT NULL,
    sale_date DATE NOT NULL
);
'''

In [6]:
details_new = '''
CREATE TABLE details_new(
    invoice VARCHAR(50) PRIMARY KEY,
    item_num INT NOT NULL,
    quantity INT NOT NULL,
    cost DECIMAL(10, 2) NOT NULL,
    retail DECIMAL(10, 2) NOT NULL
);
'''

In [7]:
store_new = '''
CREATE TABLE store_new(
    store_name VARCHAR(50) PRIMARY KEY,
    zip_code INT NOT NULL,
    store_num INT NOT NULL,
    gps TEXT NOT NULL
);
'''

In [8]:
store_location_new = '''
CREATE TABLE store_location_new(
    zip_code INT PRIMARY KEY,
    city VARCHAR(50)
);
'''

In [9]:
description_new = '''
CREATE TABLE description_new(
    item_num INT PRIMARY KEY,
    item_desc VARCHAR(50) NOT NULL,
    volume INT NOT NULL
);
'''

In [10]:
vendor_new = '''
CREATE TABLE vendor_new(
    vendor_num INT PRIMARY KEY,
    vendor_name VARCHAR(50) NOT NULL
);
'''

In [11]:
init_relations = '''
ALTER TABLE details ADD CONSTRAINT FK_details
    FOREIGN KEY (invoice) REFERENCES sale(invoice);
ALTER TABLE details ADD CONSTRAINT FK_item_num 
    FOREIGN KEY (item_num) REFERENCES description(item_num);

ALTER TABLE sale ADD CONSTRAINT FK_store_name 
    FOREIGN KEY (store_name) REFERENCES store(store_name);
ALTER TABLE sale ADD CONSTRAINT FK_vendor_num 
    FOREIGN KEY (vendor_num) REFERENCES vendor(vendor_num);

ALTER TABLE store ADD CONSTRAINT FK_zip_code 
    FOREIGN KEY (zip_code) REFERENCES store_location(zip_code);
'''

In [12]:
queries = [sale_new, details_new, store_new, store_location_new, description_new, vendor_new]
file_names = ['sale_new', 'details_new', 'store_new', 'store_location_new', 'description_new', 'vendor_new']

In [13]:
conn = psycopg2.connect(user=conn_data['USERNAME'], password=conn_data['PASSWORD'], dbname=conn_data['DB_NAME'], host=conn_data['HOST'], port=conn_data['PORT'])

for query, name in zip(queries, file_names):
    with open(f'{name}.csv') as f0:
        cur = conn.cursor()
        cur.execute(f'DROP TABLE IF EXISTS {name}')
        cur.execute(query)
        cur.copy_from(f0, f'{name}', sep=',')
    conn.commit()